In [12]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import os, pathlib

exe = pathlib.Path(os.environ["CONDA_PREFIX"]) / "Library" / "bin" / "glpsol.exe"
solver = SolverFactory("glpk", executable=str(exe))
model = ConcreteModel()

#Conjuntos
model.R = ("Alimentos Basicos", "Medicinas", "Equipos Medicos", "Agua Potable", "Mantas")
model.A = RangeSet(3)

#Parametros
P = {
    "Alimentos Basicos": 50,
    "Medicinas": 100,
    "Equipos Medicos": 120,
    "Agua Potable": 60,
    "Mantas": 40
}

S = {
    "Alimentos Basicos": 15,
    "Medicinas": 5,
    "Equipos Medicos": 20,
    "Agua Potable": 18,
    "Mantas": 10
}

V = {
    "Alimentos Basicos": 8,
    "Medicinas": 2,
    "Equipos Medicos": 10,
    "Agua Potable": 12,
    "Mantas": 6
}

W = {
    1: 30,
    2: 40,
    3: 50
}

VA = {
    1: 25,
    2: 30,
    3: 35
}

#Variable de decisión
model.x = Var(model.R, model.A, within=Binary)


#Funcion Objetivo
model.obj = Objective(expr= sum(model.x[r, a] * P[r] for r in model.R for a in model.A), sense = maximize)

#Restricciones

model.unrecursounavion = ConstraintList()
for r in model.R:
    model.unrecursounavion.add(expr= sum(model.x[r, a] for a in model.A) <= 1)



model.volumenmax = ConstraintList()
for a in model.A:
    model.volumenmax.add(expr= sum(model.x[r,a] * V[r] for r in model.R) <= VA[a])


model.pesomax = ConstraintList()
for a in model.A:
    model.pesomax.add(expr= sum(model.x[r,a] * S[r] for r in model.R) <= W[a])


solver.solve(model)
print()

print("Valor óptimo de la función objetivo =", model.obj())


print("\n=== Asignaciones óptimas ===")
for r in model.R:
    for a in model.A:
        if value(model.x[r, a]) > 0.5:  
            print(f"El recurso {r} fue asignada al avion {a}")


Valor óptimo de la función objetivo = 370.0

=== Asignaciones óptimas ===
El recurso Alimentos Basicos fue asignada al avion 2
El recurso Medicinas fue asignada al avion 2
El recurso Equipos Medicos fue asignada al avion 1
El recurso Agua Potable fue asignada al avion 2
El recurso Mantas fue asignada al avion 1


## Parte B

In [13]:
# Medicinas no pueden ir en el avión 1

model.restriccion_medicinas = Constraint(expr=model.x['Medicinas', 1] == 0)

In [14]:
# Equipos médicos y agua potable no pueden ir en el mismo avión:
def no_combo(model, a):
    return model.x["Agua Potable", a] + model.x["Equipos Medicos", a] <= 1

model.no_combo = Constraint(model.A, rule=no_combo)

In [15]:
print("\n=== Asignaciones óptimas ===")
for r in model.R:
    for a in model.A:
        if value(model.x[r, a]) > 0.5:  
            print(f"El recurso {r} fue asignada al avion {a}")


=== Asignaciones óptimas ===
El recurso Alimentos Basicos fue asignada al avion 2
El recurso Medicinas fue asignada al avion 2
El recurso Equipos Medicos fue asignada al avion 1
El recurso Agua Potable fue asignada al avion 2
El recurso Mantas fue asignada al avion 1
